# Рекомендация тарифов

__Описание проекта__

__Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.__


## Откройте и изучите файл

In [30]:
# импортирум необходимые библиотеки
import pandas as pd
from scipy import stats as st
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# на случай ошибок поставим игнор
import warnings
warnings.filterwarnings("ignore")

In [31]:
# откроем датасет и выведем первые 30 строк
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [32]:
# посмотрим общую информацию и размеры таблицы 
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


(3214, 5)

`В нашей таблице 5 столбцов и 3214 значений. Так как ранее уже делали предобработку данных, то сразу переходим к выборкам и моделям.`

`Задание: построить модель для задачи классификации, которая выберет подходящий тариф. В таблице столбец is_ultra со значениями 1 и 0. Это: 1 - да,подходит тариф Ultra; 0 - вам не подходит данный тариф. Получается качественный тип данных. И, соответственно, целевым признаком является данный столбец.`

## Разбейте данные на выборки

`Разделим набор данных на выборки:` 
* обучающую (df_train) для обучения модели(60% исходных данных);
* валидационную (df_valid) для проверки, переобучилась ли модель (20% данных);
* тестовую (df_test) для оценки качества(20%).

В random_state укажем значение 12345. 
Для начала разделим на обучающую выборку и валидационную, а затем данные разделим поровну на тестовую выборку и валидационную.`

In [33]:
# обучающая,валидационная и тестовая выборки. Выведем их размеры для самопроверки 
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df, test_size=0.5, random_state=12345)
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(1607, 5)
(1607, 5)


`Теперь,когда у нас есть три таблицы, нужно подготовить к классификации и сохранить в переменных признаки.`

In [34]:
# создаем переменные для признаков и целевого признака
# обучающий набор признаков
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra'] #целевой
# валидационный
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra'] # целевой
# тестовый
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra'] # целевой

## Исследуйте модели

Для нашего анализа рассмотрим три модели:
* дерево решений
* случайный лес
* логистическая регрессия 

__Начнем с дерева решений. Важный гиперпараметр решающего дерева — max_depth__

In [35]:
best_model = None
best_result = 0
maximum_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # создадим модель, указав max_depth=depth 
    model.fit(features_train, target_train) # обучим модель на обучающем наборе 
    predictions_valid = model.predict(features_valid) # проверим модель на валид.наборе
    result = accuracy_score(target_valid, predictions_valid) # посчитаем качество модели
    if result>best_result:
        best_model = model
        best_result = result
        maximum_depth = depth
print('При максимальной глубине =',maximum_depth, '- accuracy наилучшей модели на валидационной выборке:', best_result)


При максимальной глубине = 7 - accuracy наилучшей модели на валидационной выборке: 0.8046048537647791


`Аccuracy наилучшей модели равна 0.8046048537647791 с гиперпараметром max_depth=7`

__Одного дерева недостаточно, нужен целый лес! Рассмотрим модель Cлучайный лес__

In [36]:
best_model = None
best_result = 0
maximum_depth = 0
maximum_est = 0
for est in range(1, 10):
    for depth in range(1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth = depth) # обучите модель с заданным количеством деревьев
        model.fit(features_train, target_train) # обучите модель на обуч. выборке 
        predictions_valid = model.predict(features_valid) # проверим модель на валид.наборе
        result = accuracy_score(target_valid, predictions_valid) # посчитаем качество модели
        if result>best_result:
            best_model = model
            best_result = result
            maximum_depth = depth
            maximum_est = est
print('При максимальной глубине =',maximum_depth, 'и количестве деревьев',maximum_est, '- accuracy наилучшей модели на валидационной выборке:', best_result)

При максимальной глубине = 8 и количестве деревьев 6 - accuracy наилучшей модели на валидационной выборке: 0.8139390168014935


`Аccuracy наилучшей модели равна 0.8139390168014935 с гиперпараметром max_depth=8 и количестве деревьев=6`

__Проверим модель Логистическая регрессия__


In [37]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)# обучите модель на обуч. выборке 
predictions_valid = model.predict(features_valid) # проверим модель на валид.наборе
#dump(model, 'model_9_1.joblib')
accuracy = accuracy_score(predictions_valid, target_valid)
print('Аccuracy наилучшей модели равна', accuracy)

Аccuracy наилучшей модели равна 0.7131300560049783


__Итак, рассмотрев три модели, определили их лучшие результаты.__
* В модели дерева решений при максимальной глубине равной 7 accuracy составило 0.8046048537647791
* В модели случайного леса при максимальной глубине равной 8 и количестве деревьев равном 6 accuracy составило 0.8139390168014935
* В модели логистической регрессии аccuracy составило 0.7131300560049783

__Исходя из данных значений, видим,что модель случайного леса показала себя с лучшей стороны, поэтому именно эту модель бюудем тестировать дальше.__

## Проверьте модель на тестовой выборке

In [38]:
model = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth = depth)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result_test = accuracy_score(predictions_test, target_test)
print('Accuracy при тестовой выборке показывает значение:', result_test)

Accuracy при тестовой выборке показывает значение: 0.8730553827006845


__Тестовая выборка показала значение аccuracy равной 0.8730553827006845. Это аж 87% правильных ответов!__

## (бонус) Проверьте модели на адекватность

__Общий вывод.__
__Мы рассмотрели три модели выборок(случайного леса, логистической регрессии и дерева решений), при которых нашли самые оптимальные значения правильных ответов. В ходе анализа выявили,что модель случайного леса показало себя с лучшей стороны и поэтому именно эту модель мы протестировали на тестовой выборке. Тестовая выборка показала 87% правильных ответов. Таким образов, данную модель можно использовать для первоначальной задачи,предложения выбранных тарифов.__